In [1]:
!pip install langchain langchain_community beautifulsoup4 chromadb sentence-transformers
!pip install -U lxml

  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached chromadb-1.0.15-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)
  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached langchain_core-0.3.72-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_text_splitters-0.3.9-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.4.10-py3-none-any.whl.metadata (14 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached python_dotenv-1.1.1-p

In [2]:
import os
import time
from langchain_community.document_loaders import DirectoryLoader, BSHTMLLoader
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [3]:
# --- FASE 1: COLETA DE DADOS ---
print("1. Carregando TODOS os arquivos da documentação local. Isso pode demorar vários minutos...")

1. Carregando TODOS os arquivos da documentação local. Isso pode demorar vários minutos...


In [4]:
# Certifique-se de que o caminho para a pasta descompactada está correto.
loader = DirectoryLoader(
    path='./data/python-3.13-docs-html', # O caminho para a pasta que você criou
    glob="**/*.html",          # Pega todos os arquivos .html em todas as subpastas
    loader_cls=BSHTMLLoader,    # Usa o BeautifulSoup para ler os arquivos HTML
    show_progress=True,         # Mostra uma barra de progresso (muito útil!)
    use_multithreading=True,    # Acelera o carregamento dos arquivos
    loader_kwargs={'get_text_separator': ' '} # Argumento para o BSHTMLLoader
)

In [5]:
docs = loader.load()
print(f"✅ Carregados {len(docs)} documentos.")

100%|██████████| 163/163 [00:24<00:00,  6.76it/s]

✅ Carregados 163 documentos.


In [6]:
!tree

/bin/bash: line 1: tree: command not found


In [7]:
# --- FASE 2: PROCESSAMENTO E INDEXAÇÃO ---
print("2. Dividindo os textos em pedaços (chunks)...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200,
    add_start_index=True # Adiciona o índice inicial para referência
)

2. Dividindo os textos em pedaços (chunks)...


In [8]:
splits = text_splitter.split_documents(docs)
print(f"✅ Textos divididos em {len(splits)} chunks.")

✅ Textos divididos em 9293 chunks.


In [9]:
print("3. Criando embeddings para todos os chunks...")
model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


3. Criando embeddings para todos os chunks. ESTA É A ETAPA MAIS DEMORADA...


/tmp/ipykernel_57/2264372278.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# MUDANÇA: Usar um diretório persistente para o ChromaDB
# Isso salva o banco de dados no disco para que você não precise recriá-lo toda vez!
persist_directory = 'chroma_db_completo'
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=embeddings,
    persist_directory=persist_directory
)
print("✅ Banco de dados vetorial criado e salvo no disco!")

/opt/conda/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


✅ Banco de dados vetorial criado e salvo no disco!


In [16]:
# --- FASE 3: CONSTRUINDO A CADEIA DE RESPOSTAS (RAG) ---
prompt_template = """
Use o contexto a seguir para responder à pergunta no final.
Se você não sabe a resposta, apenas diga que não sabe, não tente inventar uma resposta.
Responda em Português do Brasil.

{context}

Pergunta: {question}
Resposta útil:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


In [17]:
llm = Ollama(model="llama3", temperature=0)

In [18]:
chain_type_kwargs = {"prompt": PROMPT}
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}), # Aumentamos para 5 chunks
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

print("\n✅ Assistente com a documentação completa pronto!")


✅ Assistente com a documentação completa pronto!


In [19]:
def perguntar_assistente(query):
    print(f"\n[?] Pergunta: {query}")
    print("... Buscando na documentação completa e gerando a resposta...")
    
    start_time = time.time()
    result = qa_chain.invoke({"query": query})
    end_time = time.time()
    
    print(f"\n[!] Resposta do Assistente (levou {end_time - start_time:.2f} segundos):")
    print(result["result"])
    print("\n--- Fontes Utilizadas ---")
    # Mostra os nomes dos arquivos HTML usados para a resposta
    sources = set([doc.metadata['source'] for doc in result['source_documents']])
    for source in sources:
        print(f"- {source}")

In [24]:
perguntar_assistente("me fale sobre as PEP's")


[?] Pergunta: me fale sobre as PEP's
... Buscando na documentação completa e gerando a resposta...


/opt/conda/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



[!] Resposta do Assistente (levou 105.65 segundos):
As PEPs (Python Enhancement Proposals) são um conjunto de documentos que propõem melhorias, mudanças ou novas funcionalidades para o Python. Elas têm como objetivo coletar opiniões da comunidade sobre um problema e documentar as decisões de design que foram adicionadas ao Python.

As PEPs são escritas por membros da comunidade Python e são revisadas e aprovadas pela mesma. Elas abordam uma ampla gama de tópicos, desde melhorias no interpretador até novos recursos na biblioteca padrão.

As PEPs incluem informações sobre o problema que está sendo abordado, as soluções propostas e as razões pelas quais elas são necessárias. Elas também documentam as decisões de design que foram tomadas e como elas afetam o Python.

Em resumo, as PEPs são um importante mecanismo para a evolução do Python, permitindo que a comunidade contribua para o desenvolvimento do linguagem e garantindo que as mudanças sejam documentadas e revisadas.

--- Fontes Util